In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
larg_leg.shape

NameError: name 'larg_leg' is not defined

In [56]:
def gen_movie(data,tt,sch1,sch2):
    '''meaningful input: top view:(0,2)  left view:(1,0)  front view:(1,2)'''
    mapp = np.zeros((tt,600,600),dtype = np.uint8)
    for t in range(tt):
        for i in range(0, data.shape[1]):  #  [0,5,19,24,10,29,15,34]:
            for j in range(-3,3):
                for k in range(-3,3):
                    if(i<180):
                        
                        if(sch1==1 and sch2==2):
                            mapp[t,int(100*data[t,i,sch1]+350)+j,int(300+100*data[t,i,sch2])+k] = 255
                        elif(sch1==1 and sch2==0):
                            mapp[t,int(100*data[t,i,sch1]+350)+j,int(100+100*data[t,i,sch2])+k] = 255
                        elif(sch1==0 and sch2==2):
                            mapp[t,int(100*data[t,i,sch1]+100)+j,int(300+100*data[t,i,sch2])+k] = 255
                    else:
                        if(sch1==1 and sch2==2):
                            mapp[t,int(100*data[t,i,sch1]+350)+j,int(300+100*data[t,i,sch2])+k] = 130
                        elif(sch1==1 and sch2==0):
                            mapp[t,int(100*data[t,i,sch1]+350)+j,int(100+100*data[t,i,sch2])+k] = 130
                        elif(sch1==0 and sch2==2):
                            mapp[t,int(100*data[t,i,sch1]+100)+j,int(300+100*data[t,i,sch2])+k] = 130
    return mapp
def play_movie(mapp,tt,speed):
    for t in range(tt):
        cv2.imshow('reconstruct_movie',mapp[t])
        if cv2.waitKey(speed) & 0xFF == ord('q'):
            break
def write_movie(mapp,tt,name):
    out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc('M','J','P','G'),100,(600,600))
    bgr = np.zeros((600,600,3),dtype = np.uint8)
    for t in range(tt): 
        bgr[:,:,0] = mapp[t]
        bgr[:,:,1] = mapp[t]
        bgr[:,:,2] = mapp[t]
        out.write(bgr)
def interpolation(data, t, leg, pos, num):
    '''pos = 0,1,2,3  t = 0 , leg = left:0, 5, 10,  right:19, 24, 29'''
    ex_point = data[t,leg+pos,:].reshape(1,3)
    next_point = data[t,leg+pos+1,:].reshape(1,3)
    lam = np.linspace(0,1,num).reshape(num,1)
    enlarge_leg = lam * ex_point + (1 - lam) * next_point
    return enlarge_leg

points = np.load('recon_points.npy')
num = 15
enlarge_leg = np.zeros((899,24,num,3))
for t in range(0,899):
    i = 0
    for leg in range(6):
        for pos in range(4):
            enlarge_leg[t,i+pos,:,:] = interpolation(points,t,leg*5,pos,num)
        i += 4
larg_leg = enlarge_leg.reshape(899,24*num,3)

In [38]:
'''top view:(0,2)'''
tt = 899
speed = 10
#mapp_t = gen_movie(larg_leg,tt,0,2)
#play_movie(mapp_t,tt,speed)
cv2.destroyAllWindows()
name = 'Recon_movie_top.avi'
write_movie(mapp_t,tt,name)

In [4]:
mapp_t.shape

(899, 800, 800)

In [18]:
'''left view:(1,0)'''
tt = 899
speed = 10
mapp_l = gen_movie(larg_leg,tt,1,0)
play_movie(mapp_l,tt,speed)
cv2.destroyAllWindows()
name = 'Recon_movie_left.avi'
#write_movie(mapp,tt,name)

In [34]:
speed = 10
play_movie(mapp,tt,speed)
cv2.destroyAllWindows()

In [19]:
'''front view:(1,2)'''
tt = 899
speed = 10
mapp_f = gen_movie(larg_leg,tt,1,2)
play_movie(mapp_f,tt,speed)
cv2.destroyAllWindows()
name = 'Recon_movie_front.avi'
#write_movie(mapp,tt,name)

In [16]:
map_sum = np.concatenate((mapp_l,mapp_t,mapp_f),axis=2)
play_movie(map_sum,tt,int(20))
cv2.destroyAllWindows()

In [17]:
std_data = np.load('std_data.npy')
num = 15
enlarge_leg = np.zeros((899,24,num,3))
for t in range(0,899):
    i = 0
    for leg in [0,5,10,19,24,29]:
        for pos in range(4):
            enlarge_leg[t,i+pos,:,:] = interpolation(std_data,t,leg,pos,num)
        i += 4
larg_leg = enlarge_leg.reshape(899,24*num,3)

In [43]:
#map_sum_org = np.concatenate((mapp_l,mapp_t,mapp_f),axis=2)
play_movie(map_sum_org,tt,int(10))
cv2.destroyAllWindows()


In [63]:
map_org_recon = np.concatenate((map_sum_org, map_sum),axis=1)
play_movie(map_org_recon,tt,int(10))
cv2.destroyAllWindows()

In [91]:
def write_movie_(mapp,tt,name):
    out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc('M','J','P','G'),50,(1800,1200))
    bgr = np.zeros((1200,1800,3),dtype = np.uint8)
    for t in range(tt): 
        bgr[:,:,0] = np.floor(0.7*mapp[t]).astype(np.uint8)
        bgr[:,:,1] = np.floor(0.7*mapp[t]).astype(np.uint8)
        bgr[:,:,2] = np.floor(0.9*mapp[t]).astype(np.uint8)
        out.write(bgr)
tt=899
name = 'org_recon_moive_0.5slow.avi'#map_org_recon[:,:800,:800]
write_movie_(map_org_recon,tt,name)

In [62]:
map_org_recon.shape

(899, 1200, 1600)

In [ ]:
name = 'org_recon_moive.avi'
write_movie(map_org_recon,tt,name)